In [46]:
from BooleanRetrieval.utils import generate_incidence_matrix, generate_inverted_index, boolean_and, \
    boolean_or, boolean_not
from BooleanRetrieval.inverted_index import InvertedIndex
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# <div style="font-family: 'Garamond', serif; font-size: 22px; color: #ffffff; background-color: #34568B; text-align: center; padding: 15px; border-radius: 10px; border: 2px solid #FF6F61; box-shadow: 0 6px 12px rgba(0, 0, 0, 0.3); margin-bottom: 20px;">1. An example information retrieval problem

## <div style="font-family: 'Lucida Sans Unicode', sans-serif; font-size: 18px; color: #4A235A; background-color: #D7BDE2; text-align: left; padding: 10px; border-left: 5px solid #7D3C98; box-shadow: 0 4px 6px rgba(0, 0, 0, 0.2); margin-bottom: 10px;">1.1 Note</div>

1. **簡單的檢索方式**：傳統上，檢索可以透過逐行掃描所有文件來查找包含特定關鍵字的文檔，例如查找同時包含 "Brutus" 和 "Caesar"，但不包含 "Calpurnia" 的劇本。這種掃描類似於 Unix 的 `grep` 命令，適合小型資料集。

2. **倒排索引的需求**：隨著數據規模的增長，線性掃描已經不足以應付海量的文件集合，因此需要更高效的方法，如倒排索引，來加速檢索並支持更多樣化的查詢需求，如詞距查詢或排序檢索。

3. **布林檢索模型**：布林檢索允許將詞彙透過邏輯運算（AND、OR、NOT）組合來查找文件。在例子中，若要查詢 "Brutus AND Caesar AND NOT Calpurnia"，可使用每個詞的二進位向量，進行按位運算來確定符合條件的文件。

4. **索引結構**：倒排索引包含一個字典（或詞彙表），儲存每個詞項出現的文件列表（postings list）。字典的每個條目指向一個文檔列表，這些列表可按文檔 ID 排序，用來加速查詢。

5. **稀疏矩陣問題**：構建巨大的詞項-文檔矩陣會產生大量的 0，因為大多數文檔只包含少量的詞。因此，應該僅記錄出現的詞和文檔，避免記錄過多的空數據，這是倒排索引的核心概念。

6. **檢索系統的評估**：檢索系統的效能通常通過兩個指標來衡量：**Precision**（精確度）和 **Recall**（召回率），分別表示返回結果中相關文檔的比例，以及所有相關文檔中成功檢索到的比例。

# <div style="font-family: 'Garamond', serif; font-size: 22px; color: #ffffff; background-color: #34568B; text-align: center; padding: 15px; border-radius: 10px; border: 2px solid #FF6F61; box-shadow: 0 6px 12px rgba(0, 0, 0, 0.3); margin-bottom: 20px;">2. A first take at building an inverted index</div>

## <div style="font-family: 'Lucida Sans Unicode', sans-serif; font-size: 18px; color: #4A235A; background-color: #D7BDE2; text-align: left; padding: 10px; border-left: 5px solid #7D3C98; box-shadow: 0 4px 6px rgba(0, 0, 0, 0.2); margin-bottom: 10px;">2.1 Note</div>

這段文本描述了如何透過排序和分組來建立倒排索引的過程，具體步驟如下：

1. **收集文件**：首先，收集需要建立索引的文件集，這裡舉例使用了莎士比亞的文本。

2. **標記化（Tokenization）**：將每個文件轉換成詞彙列表（tokens），例如將句子 "Friends, Romans, countrymen. So let it be with Caesar..." 轉換成 "Friends Romans countrymen So let it be with Caesar"。

3. **語言預處理**：將詞彙標準化，去掉大小寫區分，轉換成基礎形式（如單數化等），使其成為索引用的標準詞彙。例如 "Friends" 變為 "friend"，"Romans" 變為 "roman"。

4. **倒排索引構建**：根據每個詞彙的出現次數，為其建立一個索引，包括文檔的唯一編號（docID）。這個過程將詞彙按字母順序排序，將出現相同詞彙的文檔ID分組，生成包含詞彙及其對應文檔列表的倒排索引。

5. **統計與優化**：在建立索引時，還會記錄一些統計信息，如每個詞彙的文檔頻率（document frequency），這些信息雖然對基礎的布林檢索引擎不是必需的，但能提高檢索效率，並且對後續的排序檢索模型有用。Postings List 會根據文檔ID排序，從而使得查詢過程更高效。

### 倒排索引的構成：
- **字典（Dictionary）**：儲存詞彙及指向每個詞彙 Postings List 的指針。
- **Postings List**：記錄出現該詞彙的文檔ID，並可以擴展儲存該詞在文檔中的詞頻或出現位置。

倒排索引是最有效的支持即席（ad hoc）文本檢索的結構，通過優化字典和 Postings List 的儲存方式，可以提高檢索性能和儲存效率。

這個過程的關鍵點是透過排序和分組來將相同詞彙出現的文檔ID整理在一起，從而減少重複數據，並優化查詢時間。

## <div style="font-family: 'Lucida Sans Unicode', sans-serif; font-size: 18px; color: #4A235A; background-color: #D7BDE2; text-align: left; padding: 10px; border-left: 5px solid #7D3C98; box-shadow: 0 4px 6px rgba(0, 0, 0, 0.2); margin-bottom: 10px;">2.2 Exercise</div>

### Draw the inverted index that would be built for the following document collection.
(See Figure 1.3 for an example.)
 - Doc 1 new home sales top forecasts
 - Doc 2 home sales rise in july
 - Doc 3 increase in home sales in july
 - Doc 4 july new home sales rise

In [47]:
# Document collection
documents = {
    "Doc 1": "new home sales top forecasts",
    "Doc 2": "home sales rise in july",
    "Doc 3": "increase in home sales in july",
    "Doc 4": "july new home sales rise"
}

index = InvertedIndex()

# Step-by-step process to build the inverted index
for doc_id, text in documents.items():
    index.add_document(doc_id, text)

# Display the sorted index
sorted_index = index.sort_and_group()
for term, postings in sorted_index.items():
    print(f"Term: {term}, Postings: {postings}")

Term: forecasts, Postings: [('Doc 1', 1)]
Term: home, Postings: [('Doc 1', 1), ('Doc 2', 1), ('Doc 3', 1), ('Doc 4', 1)]
Term: in, Postings: [('Doc 2', 1), ('Doc 3', 2)]
Term: increase, Postings: [('Doc 3', 1)]
Term: july, Postings: [('Doc 2', 1), ('Doc 3', 1), ('Doc 4', 1)]
Term: new, Postings: [('Doc 1', 1), ('Doc 4', 1)]
Term: rise, Postings: [('Doc 2', 1), ('Doc 4', 1)]
Term: sales, Postings: [('Doc 1', 1), ('Doc 2', 1), ('Doc 3', 1), ('Doc 4', 1)]
Term: top, Postings: [('Doc 1', 1)]


### Consider these documents:
- Doc 1 breakthrough drug for schizophrenia
- Doc 2 new schizophrenia drug
- Doc 3 new approach for treatment of schizophrenia
- Doc 4 new hopes for schizophrenia patients

1. Draw the term-document incidence matrix for this document collection
2. Draw the inverted index representation for this collection, as 
```
word -> ["Doc 1", "Doc 3"]
```

In [48]:
# Document collection
documents = {
    "Doc 1": "breakthrough drug for schizophrenia",
    "Doc 2": "new schizophrenia drug",
    "Doc 3": "new approach for treatment of schizophrenia",
    "Doc 4": "new hopes for schizophrenia patients"
}

# Initialize the inverted index
index = InvertedIndex()

# Add documents to the inverted index
for doc_id, text in documents.items():
    index.add_document(doc_id, text)

# Generate the Term-Document Incidence Matrix
generate_incidence_matrix(index, documents)

# Generate the Inverted Index Representation
generate_inverted_index(index)

Term-Document Incidence Matrix:
Term            Doc 1 Doc 2 Doc 3 Doc 4
approach        0 0 1 0
breakthrough    1 0 0 0
drug            1 1 0 0
for             1 0 1 1
hopes           0 0 0 1
new             0 1 1 1
of              0 0 1 0
patients        0 0 0 1
schizophrenia   1 1 1 1
treatment       0 0 1 0

Inverted Index Representation:
approach -> ['Doc 3']
breakthrough -> ['Doc 1']
drug -> ['Doc 1', 'Doc 2']
for -> ['Doc 1', 'Doc 3', 'Doc 4']
hopes -> ['Doc 4']
new -> ['Doc 2', 'Doc 3', 'Doc 4']
of -> ['Doc 3']
patients -> ['Doc 4']
schizophrenia -> ['Doc 1', 'Doc 2', 'Doc 3', 'Doc 4']
treatment -> ['Doc 3']


### For the document collection shown in Exercise 1.2, what are the returned results for
these queries:
1. schizophrenia AND drug
2. for AND NOT(drug OR approach)

In [49]:
# 1. schizophrenia AND drug
result_1 = boolean_and("schizophrenia", "drug", index)

# 2. for AND NOT (drug OR approach)
# First, perform the OR operation for (drug OR approach)
or_result = boolean_or("drug", "approach", index)

# Now, perform the NOT operation for "for" AND NOT (drug OR approach)
result_2 = boolean_not("for", or_result, index)

In [50]:
print("Results for the queries:")
print("1. 'schizophrenia AND drug':", result_1)
print("2. 'for AND NOT(drug OR approach)':", result_2)

Results for the queries:
1. 'schizophrenia AND drug': ['Doc 1', 'Doc 2']
2. 'for AND NOT(drug OR approach)': ['Doc 4']


# <div style="font-family: 'Garamond', serif; font-size: 22px; color: #ffffff; background-color: #34568B; text-align: center; padding: 15px; border-radius: 10px; border: 2px solid #FF6F61; box-shadow: 0 6px 12px rgba(0, 0, 0, 0.3); margin-bottom: 20px;">3. Processing boolean queries</div>

## <div style="font-family: 'Lucida Sans Unicode', sans-serif; font-size: 18px; color: #4A235A; background-color: #D7BDE2; text-align: left; padding: 10px; border-left: 5px solid #7D3C98; box-shadow: 0 4px 6px rgba(0, 0, 0, 0.2); margin-bottom: 10px;">3.1 Note</div>

### 使用倒排索引進行布林檢索的查詢處理

在布林檢索系統中，透過倒排索引處理查詢可以有效提高檢索效率。以下是如何處理這類查詢的總結：

#### 1. **簡單的合取查詢 (AND 查詢)**

以查詢 **"Brutus AND Calpurnia"** 為例，處理步驟如下：
1. 在字典中找到 "Brutus"。
2. 取得 "Brutus" 的 Postings List（文檔列表）。
3. 在字典中找到 "Calpurnia"。
4. 取得 "Calpurnia" 的 Postings List。
5. 將兩個 Postings List 進行交集運算，找到同時包含 "Brutus" 和 "Calpurnia" 的文檔。

這個交集運算是檢索的核心，通過合併兩個排序過的 Postings List 來快速找到同時包含兩個詞的文檔。合併過程中，對比兩個列表中的文檔ID，若相同則將其加入結果，否則前進較小的文檔ID。這種方法的時間複雜度是 O(x + y)，其中 x 和 y 是兩個 Postings List 的長度。

#### 2. **複雜查詢 (OR 和 NOT 操作)**

例如查詢 **"(Brutus OR Caesar) AND NOT Calpurnia"**，需要進行以下步驟：
1. 先分別取得 "Brutus" 和 "Caesar" 的 Postings List，並進行 OR 操作，合併這兩個列表。
2. 取得 "Calpurnia" 的 Postings List，然後將前一步的結果與 "Calpurnia" 的結果進行 AND NOT 操作，排除包含 "Calpurnia" 的文檔。

#### 3. **查詢優化**

為了提高處理查詢的效率，布林檢索系統會根據詞頻（每個詞出現的文檔數量）來決定處理順序。對於查詢如 **"Brutus AND Caesar AND Calpurnia"**，先處理詞頻最低的詞能減少中間結果的大小，進而減少總運算量。

#### 4. **處理複雜查詢**

對於更複雜的查詢，如 **"(madding OR crowd) AND (ignoble OR strife) AND (killed OR slain)"**，系統會先根據每個詞的出現頻率預估 OR 操作後列表的大小，並按從小到大的順序進行查詢處理，這樣可以最小化總運算量。

# <div style="font-family: 'Garamond', serif; font-size: 22px; color: #ffffff; background-color: #34568B; text-align: center; padding: 15px; border-radius: 10px; border: 2px solid #FF6F61; box-shadow: 0 6px 12px rgba(0, 0, 0, 0.3); margin-bottom: 20px;">4. The extended Boolean model</div>

## <div style="font-family: 'Lucida Sans Unicode', sans-serif; font-size: 18px; color: #4A235A; background-color: #D7BDE2; text-align: left; padding: 10px; border-left: 5px solid #7D3C98; box-shadow: 0 4px 6px rgba(0, 0, 0, 0.2); margin-bottom: 10px;">4.1 Note</div>

### 擴展布林模型與排序檢索對比

布林檢索模型與排序檢索模型，如向量空間模型，形成鮮明對比。在排序檢索模型中，用戶只需輸入自由文本查詢，而不需要使用精確的運算符語言來組建查詢表達式，系統會自動決定哪些文檔最符合查詢。在學術界對排序檢索的優勢進行了數十年的研究，但直到20世紀90年代，布林檢索模型仍是大多數大型商業信息提供商的主要或唯一檢索選項。

### 布林檢索的擴展模型

傳統的布林檢索模型只包含基本的運算（如 AND、OR 和 NOT），這對於許多信息需求來說太過有限。因此，這些系統實施了擴展的布林檢索模型，通過加入額外的運算符，如**近鄰運算符**，來提高檢索的靈活性。近鄰運算符允許指定查詢中的兩個詞必須在文檔中相互靠近，這種靠近可以根據字數限制或結構單位（如句子或段落）來測量。

### 商業布林檢索範例：Westlaw

Westlaw 是最大的商業法律檢索服務，支持布林檢索和排序檢索。布林檢索提供精確的控制，能夠滿足專業領域的高精度需求，特別是在法律檢索中。專業用戶偏愛布林模型，因為它能夠精確地控制檢索結果。然而，研究表明，在許多情況下，排序檢索比布林檢索能夠提供更好的結果。

### 布林檢索的局限性

布林檢索的主要問題在於使用 AND 運算符容易導致高精度但低召回率的檢索，而使用 OR 則可能導致低精度但高召回率的檢索，難以找到滿意的中間方案。此外，布林模型僅記錄詞的出現與否，無法根據詞在文檔中出現的頻率來進行加權檢索。

## <div style="font-family: 'Lucida Sans Unicode', sans-serif; font-size: 18px; color: #4A235A; background-color: #D7BDE2; text-align: left; padding: 10px; border-left: 5px solid #7D3C98; box-shadow: 0 4px 6px rgba(0, 0, 0, 0.2); margin-bottom: 10px;">4.2 Exercise</div>

### Write a query using Westlaw syntax which would find any of the words professor,
teacher, or lecturer in the same sentence as a form of the verb explain

Westlaw Query Syntax
```
(professor OR teacher OR lecturer) /s explain!
```

### Try using the Boolean search features on a couple of major web search engines. 

For instance, choose a word, such as burglar, and submit the queries (i) burglar, (ii) burglar
AND burglar, and (iii) burglar OR burglar. Look at the estimated number of results and
top hits. 

Do they make sense in terms of Boolean logic? Often they haven’t for major
search engines. 

Can you make sense of what is going on? What about if you try
different words? For example, query for (i) knight, (ii) conquer, and then (iii) knight OR
conquer. What bound should the number of results from the first two queries place
on the third query? 

Is this bound observed?

1. **burglar**: 
    - (i) "burglar": 34,400,000 results
    - (ii) 
        - "burglar" & "burglar": 35,000,000 results
        - "burglar" AND "burglar": 35,100,000 results
    - (iii) 
        - "burglar" | "burglar": 35,300,000 results
        - "burglar" OR "burglar": 35,400,000 results

2. **knight**:
    - (i) "knight": 572,000,000 results
    - (ii) "conquer": 129,000,000 results
    - (iii) 
        - "knight" | "conquer": 744,000,000 results
        - "knight" OR "conquer": 737,000,000 results

